<a href="https://colab.research.google.com/github/Martinccv/Clases-DS/blob/main/Clase28_NLP_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP

En esta clase aprendimos los siguientes conceptos:

- Stop words
- Palabra raiz (Lemma)
- Tokenizado
- Vectorizado (BOW / TFIDF)

Ahora veremos como hacer esto en python.

Para NLP introduciremos algunas librerías nuevas, una de ellas es [Spacy](https://spacy.io/).

Otras librerías conocidas son:
- nltk
- gensim

Y una librería que es de lo mejor que hay en NLP actualmente: Hugging face.

Comenzaremos esta clase con Spacy. Ya viene pre-instalada en google colab por lo que no será necesario instalarla. Si luego trabajan en algún entorno en el que no este instalada, pueden seguir el tutorial de la página oficial.

Para usar spacy, debemos descargar un modelo del lenguaje que vayamos a trabajar. En este caso estaremos trabajando con textos en inglés (ya está descargado en colab), pero si en otro momento utilizan otro idioma, deben descargarlo desde https://spacy.io/models.

Ahora, importemos spacy y carguemos el modelo en inglés que utilizaremos en este notebook:




In [2]:
import spacy
nlp = spacy.load("en_core_web_sm")

### Carga de datos

El siguiente comando descargará un dataset de reviews de películas (en inglés) en su entorno de colab.

Luego de correr la siguiente celda, deberían ver en su entorno el directorio "acllmdb" que dentro contiene los datos.



In [3]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar xvzf /content/aclImdb_v1.tar.gz

Se truncaron las últimas líneas 5000 del resultado de transmisión.
aclImdb/train/unsup/44983_0.txt
aclImdb/train/unsup/44982_0.txt
aclImdb/train/unsup/44981_0.txt
aclImdb/train/unsup/44980_0.txt
aclImdb/train/unsup/44979_0.txt
aclImdb/train/unsup/44978_0.txt
aclImdb/train/unsup/44977_0.txt
aclImdb/train/unsup/44976_0.txt
aclImdb/train/unsup/44975_0.txt
aclImdb/train/unsup/44974_0.txt
aclImdb/train/unsup/44973_0.txt
aclImdb/train/unsup/44972_0.txt
aclImdb/train/unsup/44971_0.txt
aclImdb/train/unsup/44970_0.txt
aclImdb/train/unsup/44969_0.txt
aclImdb/train/unsup/44968_0.txt
aclImdb/train/unsup/44967_0.txt
aclImdb/train/unsup/44966_0.txt
aclImdb/train/unsup/44965_0.txt
aclImdb/train/unsup/44964_0.txt
aclImdb/train/unsup/44963_0.txt
aclImdb/train/unsup/44962_0.txt
aclImdb/train/unsup/44961_0.txt
aclImdb/train/unsup/44960_0.txt
aclImdb/train/unsup/44959_0.txt
aclImdb/train/unsup/44958_0.txt
aclImdb/train/unsup/44957_0.txt
aclImdb/train/unsup/44956_0.txt
aclImdb/train/unsup/44955_0.txt
aclIm

Si navegan el directorio que tiene los datos, veran que hay un directorio train y otro test.

A su vez, dentro de cada uno de ellos podrán ver los directorios neg y pos. Ahi se encuentran los datos que utilizaremos hoy.

En neg se encuentran reviews negativas, en pos review positivas.

La siguiente celda, lista los nombres de los archivos que hay en /content/aclImdb/test/pos.

Ven algo raro?


In [4]:
!ls /content/aclImdb/test/pos

0_10.txt      11606_8.txt   1964_8.txt	 3573_9.txt   5179_10.txt  678_7.txt	8394_9.txt
10000_7.txt   11607_7.txt   1965_10.txt  3574_9.txt   5180_8.txt   6788_9.txt	8395_10.txt
10001_9.txt   11608_9.txt   1966_9.txt	 3575_7.txt   518_10.txt   6789_10.txt	8396_10.txt
10002_8.txt   11609_10.txt  1967_10.txt  3576_7.txt   5181_10.txt  6790_9.txt	8397_10.txt
10003_8.txt   11610_10.txt  1968_8.txt	 3577_8.txt   5182_10.txt  679_10.txt	8398_7.txt
10004_9.txt   116_10.txt    1969_10.txt  357_7.txt    5183_9.txt   6791_10.txt	8399_8.txt
10005_8.txt   1161_10.txt   196_9.txt	 3578_9.txt   5184_8.txt   6792_9.txt	8400_7.txt
10006_7.txt   11611_7.txt   1970_8.txt	 3579_8.txt   5185_7.txt   6793_10.txt	8401_10.txt
10007_10.txt  11612_10.txt  197_10.txt	 3580_10.txt  5186_7.txt   6794_10.txt	8402_7.txt
10008_8.txt   11613_7.txt   1971_7.txt	 3581_10.txt  5187_8.txt   6795_10.txt	8403_7.txt
10009_10.txt  11614_9.txt   1972_7.txt	 3582_10.txt  5188_7.txt   6796_10.txt	8404_10.txt
1000_9.txt    11615_

Podemos ver que cada review está en un archivo .txt distinto. ¿Cómo podemos leer este tipo de archivos en pandas? Hasta ahora veníamos levantando únicamente CSVs.

En python, se pueden abrir achivos con el comando:



```
with open("ruta_al_archivo", modo_de_lectura) as f:
  # Acá ya tenemos acceso al archivo con el nombre f
```

donde modo_de_lectura puede ser:

- r: read
- w: write
- a: append

Más detalles: https://www.w3schools.com/python/ref_func_open.asp

Además, si importamos el paquete

```
import os
```

podremos utilizar una función para listar el nomrbe de todos los archivos que se encuentran en un directorio:

```
os.listdir("directorio")
```

Entonces, lo que vamos a hacer es abrir el directorio donde se encuentran los archivos y leerlos uno por uno. A todo esto lo guardaremos luego en un dataframe de pandas.

Ejemplo de listado de reviews negativas con os.listdir:

In [5]:
import os

os.listdir("/content/aclImdb/train/neg")

['539_4.txt',
 '625_4.txt',
 '4459_1.txt',
 '10352_4.txt',
 '4243_2.txt',
 '5664_2.txt',
 '3690_1.txt',
 '3077_1.txt',
 '7293_3.txt',
 '8870_4.txt',
 '7186_2.txt',
 '8279_4.txt',
 '10530_4.txt',
 '1951_1.txt',
 '9739_4.txt',
 '4609_4.txt',
 '9330_2.txt',
 '7427_3.txt',
 '11889_1.txt',
 '11161_4.txt',
 '536_4.txt',
 '5805_1.txt',
 '6058_1.txt',
 '12351_4.txt',
 '2119_4.txt',
 '523_1.txt',
 '1862_1.txt',
 '2714_4.txt',
 '5107_1.txt',
 '9694_4.txt',
 '10072_1.txt',
 '720_4.txt',
 '7050_3.txt',
 '10628_1.txt',
 '5115_1.txt',
 '3307_3.txt',
 '5157_1.txt',
 '9770_1.txt',
 '10230_1.txt',
 '9651_3.txt',
 '4289_3.txt',
 '10796_1.txt',
 '8543_3.txt',
 '4470_3.txt',
 '4883_1.txt',
 '4208_3.txt',
 '4490_1.txt',
 '611_4.txt',
 '5361_1.txt',
 '5487_4.txt',
 '4979_3.txt',
 '7014_3.txt',
 '5759_3.txt',
 '6173_1.txt',
 '11581_2.txt',
 '12068_1.txt',
 '12338_1.txt',
 '3446_1.txt',
 '1804_2.txt',
 '5777_4.txt',
 '4988_2.txt',
 '6789_4.txt',
 '8003_3.txt',
 '571_1.txt',
 '11158_1.txt',
 '7340_1.txt',
 '11

Ahora, con un bucle for leemos todas las reviews negativas y las guardamos en una lista:

In [6]:
dir_neg_train = "/content/aclImdb/train/neg"
neg_reviews = []

for f in os.listdir(dir_neg_train):
  with open(f"{dir_neg_train}/{f}", encoding='utf-8') as neg:
    neg_reviews.append(neg.read())

Imprimimos las primeras 3 para corroborar que nuestro código funcione bien:

In [7]:
neg_reviews[:3]

['Around the late 1970\'s, animator Don Bluth, frustrated with the output his company, Disney was churning, defected from the Mouse House to form his own studio. His first production, THE SECRET OF NIMH, was a brilliant feature that still holds up well to this day. This was followed by AN American TAIL and THE LAND BEFORE TIME, both of which were made under the involvement of Steven Spielberg and were commercially successful. Although none of those two films had the dark adult appeal of NIMH, they still are very charming, enjoyable features for both children and grown-ups. But before long, Don Bluth had his first major misfire with ALL DOGS GO TO HEAVEN; critics were especially harsh on this film, and matters weren\'t helped by the fact that it opened alongside Disney\'s THE LITTLE MERMAID.<br /><br />Considering that the movie has such a friendly-sounding title, one would expect ALL DOGS GO TO HEAVEN to be pleasant family fare. Instead Bluth provides a surprisingly dark story involvin

In [8]:
len(neg_reviews)

12500

Ahora, debemos hacer lo mismo con las negativas de test y luego con las positivas.

En nuestro caso, vamos a hacer nuestro propio train/test split, por lo que las a reviews que están en el directorio de test las guardaremos en la misma lista que recién.

Agregar a la lista "neg_reviews" las reviews negativas de test:

In [9]:
dir_neg_test = "/content/aclImdb/test/neg"
for f in os.listdir(dir_neg_test):
  with open(f"{dir_neg_test}/{f}",encoding='utf-8') as neg:
    neg_reviews.append(neg.read())

Imprimir el largo de la nueva lista para corroborar que se hayan agregado todas las reviews (deberían tener 25mil)

In [10]:
#COMPLETAR
len(neg_reviews)

25000

Ahora, hacer lo mismo con las pos:

In [11]:
#COMPLETAR
dir_pos_train = "/content/aclImdb/train/pos"
dir_pos_test = "/content/aclImdb/test/pos"
pos_reviews = []
for f in os.listdir(dir_pos_train):
  with open(f"{dir_pos_train}/{f}", encoding='utf-8') as pos:
    pos_reviews.append(pos.read())

for f in os.listdir(dir_pos_test):
  with open(f"{dir_pos_test}/{f}",encoding='utf-8') as pos:
    pos_reviews.append(pos.read())

print(len(pos_reviews))
print(pos_reviews[:3])

25000
['I have to say that this TV movie was the work that really showed how talented Melissa Joan Hart is. We are so used to, now, seeing her in a sitcom and I really hope that a TV station will show this TV movie again soon as it will show the Sabrina fans that MJH shines in a drama. Seen as we have watched her on Sabrina now for now 5 years and so to give the viewers a taste of her much unused talent would be a plus. Melissa plays her role so well in this wanting her parents "done away" with so she can be with the guy she loves. One thing that all Sabrina viewers will notice, Melissa works with David Lascher in this, well before he took the role of Josh on Sabrina. So it would be kind of neat to see this currently whenever it gets aired again. Hopefully MJH gets some good roles in movies or even in more TV Movies, sort of like Kellie Martin who has always shined in TV Movies. Lots of unused talent waiting to bust out when it comes to Melissa Joan Hart, you shine always Melissa!!!', 

Deberían tener 25mil reviews de cada tipo.

Ahora, almacenaremos estos datos en un dataframe de pandas para trabajar de forma más simple.

In [12]:
import pandas as pd
pos_df = pd.DataFrame(pos_reviews, columns=["REVIEW"])
pos_df["TARGET"] = "POS"
neg_df = pd.DataFrame(neg_reviews, columns=["REVIEW"])
neg_df["TARGET"] = "NEG"

df = pd.concat([pos_df, neg_df], axis="rows")
df.head()

,REVIEW,TARGET
0,I have to say that this TV movie was the work ...,POS
1,"Shannon Lee,the daughter of Bruce Lee,delivers...",POS
2,i LOVED IT and was SO shattered that there not...,POS
3,I just saw a press screening of this film and ...,POS
4,Over-powered mobile suits that can annihilate ...,POS


In [13]:
df.shape

(50000, 2)

In [14]:
df.sample(5)

,REVIEW,TARGET
12889,""" I have wrestled with death. It is the most u...",POS
15317,"""Graduation Day"" was released in May 1981, dur...",NEG
406,"Hilarious, laugh out loud moments ... and yet ...",NEG
19976,"And again, Columbia Pictures decides to merely...",NEG
20616,Boogie Nights is one of the best films to come...,POS


In [23]:
df.describe()

,REVIEW,TARGET
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,POS
freq,5,25000


Ya tenemos nuestro dataframe listo para trabajar.

### Spacy

Como dijimos anteriormente, trabajaremos con la librería spacy. Siempre debemos importarla e instanciarla llamando a nuestro lenguaje. Si queremos instanciarla con un lenguaje que no tenemos descargado, nos dará un error y ahi podemos copiar y pegar el código para descargar el mismo.

In [15]:
import spacy
nlp = spacy.load("en_core_web_sm")

Para tokenizar un texto en spacy, simplemente utilizamos el objeto que instanciamos (que en este caso llamamos nlp)

Por ejemplo:

In [16]:
nlp("Hola como estás")

Hola como estás

Si queremos acceder a cada uno de los tokens, podemos utilizar por ejemplo un bucle for:

In [17]:
for token in nlp("Hola como estás"):
  print(token)
  print("---")

Hola
---
como
---
estás
---


### Stop words

En spacy, tenemos para cada idioma un listado de stop words por defecto (que podemos modificar agregando o quitando las que necesitemos).

Ejecutando la siguiente celda, podemos ver el listado que viene por defecto para el idioma inglés.

In [18]:
nlp.Defaults.stop_words

{"'d",
 "'ll",
 "'m",
 "'re",
 "'s",
 "'ve",
 'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'both',
 'bottom',
 'but',
 'by',
 'ca',
 'call',
 'can',
 'cannot',
 'could',
 'did',
 'do',
 'does',
 'doing',
 'done',
 'down',
 'due',
 'during',
 'each',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'empty',
 'enough',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'everywhere',
 'except',
 'few',
 'fifteen',
 'fifty',
 'first',
 'five',
 'for',
 'former',
 'formerly',
 'forty',
 'four',
 'from',
 'fron

Si queremos agregar una stopword, podemos hacerlo con el método .add() de las listas.

Por ejemplo, imaginen que queremos agregar la palabra "test".

Primero validamos si existe en la lista:


In [19]:
"test" in nlp.Defaults.stop_words

False

No existe, la agreguemos:

In [20]:
nlp.Defaults.stop_words.add("test")

In [21]:
"test" in nlp.Defaults.stop_words

True

Ahora si existe.

Para saber si un token es una stopword o no, podemos utilizar el atributo is_stop de un token.

Veamos un ejemplo:

In [22]:
for token in nlp("My name is Hugo. I am 38 years old and I live in Córdoba."):
  if token.is_stop:
    print(f"La palabra: {token.text} es una stop word.")

La palabra: My es una stop word.
La palabra: name es una stop word.
La palabra: is es una stop word.
La palabra: I es una stop word.
La palabra: am es una stop word.
La palabra: and es una stop word.
La palabra: I es una stop word.
La palabra: in es una stop word.


De esta forma, podemos limpiar las stop words de un texto. Veamos un ejemplo en el que limpiamos las stop words del texto "My name is Alexis. I am 25 years old and I live in BsAs.".

In [24]:
def clean_stop_words(text):
  clean_text = []
  for token in nlp(text):
    if not token.is_stop:
      clean_text.append(token.text)

  return " ".join(clean_text)

In [25]:
texto = "My name is Hugo. I am 38 years old and I live in Córdoba."

clean_stop_words(texto)

'Hugo . 38 years old live Córdoba .'

Vemos que nos limpio las stop words, pero además necesitaríamos pasar el texto a minúsculas y eliminar los signos de puntuación.

Para lo primero, podemos utilizar la función lower() de python.

Para lo segundo, los tokens tienen el atributo token.is_punct.

EJERCICIO: Crear una nueva función (basada en la que definimos recien) que se llame clean_text y además de eliminar stop words, elimine signos de puntuación y convierta todo a minúsculas.

In [27]:
#COMPLETAR
def clean_text(text):
  clean_text = []
  for token in nlp(text):
    if not token.is_punct and not token.is_stop:
      clean_text.append(token.text.lower())

  return " ".join(clean_text)

Probamos la función con el mismo texto que recién:

In [28]:
texto = "My name is Hugo. I am 38 years old and I live in Córdoba."

clean_text(texto)

'hugo 38 years old live córdoba'

Ahora, si quisiéramos aplicar esta función a nuestro dataset entero, como lo haríamos???

### Raiz (lemma)

En spacy, también podemos llevar a las palabras a su raiz de una forma muy simple utilizando el atributo .lemma_ (recuerden que finaliza con _) de los tokens.

Veamos un ejemplo:

In [29]:
text = """Reeves look like an Oscar winner this film bites (pun not intended).
The best thing about it is the box of eRATicate in the 2nd segment"""

for token in nlp(text):
    print(f"ORIGINAL {token.text}, LEMMA: {token.lemma_}")

ORIGINAL Reeves, LEMMA: reeve
ORIGINAL look, LEMMA: look
ORIGINAL like, LEMMA: like
ORIGINAL an, LEMMA: an
ORIGINAL Oscar, LEMMA: Oscar
ORIGINAL winner, LEMMA: winner
ORIGINAL this, LEMMA: this
ORIGINAL film, LEMMA: film
ORIGINAL bites, LEMMA: bite
ORIGINAL (, LEMMA: (
ORIGINAL pun, LEMMA: pun
ORIGINAL not, LEMMA: not
ORIGINAL intended, LEMMA: intend
ORIGINAL ), LEMMA: )
ORIGINAL ., LEMMA: .
ORIGINAL 
, LEMMA: 

ORIGINAL The, LEMMA: the
ORIGINAL best, LEMMA: good
ORIGINAL thing, LEMMA: thing
ORIGINAL about, LEMMA: about
ORIGINAL it, LEMMA: it
ORIGINAL is, LEMMA: be
ORIGINAL the, LEMMA: the
ORIGINAL box, LEMMA: box
ORIGINAL of, LEMMA: of
ORIGINAL eRATicate, LEMMA: eRATicate
ORIGINAL in, LEMMA: in
ORIGINAL the, LEMMA: the
ORIGINAL 2nd, LEMMA: 2nd
ORIGINAL segment, LEMMA: segment


EJERCICIO: A la función clean text, agregarle que convierta el texto a lemma. Quitar puntuación.

In [30]:
#COMPLETAR
def clean_text(text):
  clean_text = []
  for token in nlp(text):
    if not token.is_punct and not token.is_stop:
      clean_text.append(token.lemma_.lower())

  return " ".join(clean_text)

In [31]:
text = """Reeves look like an Oscar winner this film bites (pun not intended).
The best thing about it is the box of eRATicate in the 2nd segment ? ! """
clean_text(text)

'reeve look like oscar winner film bite pun intend \n good thing box eraticate 2nd segment'

Vemos que todavía quedan caracteres especiales como por ejemplo \n. Ya veremos en próximas clases como limpiar este tipo de elementos utilizando expresiones regulares.

Por ahora, podemos utilizar la función .replace() de los strings. Por ejemplo:

In [32]:
clean_text(text).replace("\n", "")

'reeve look like oscar winner film bite pun intend  good thing box eraticate 2nd segment'

### Bag of words

Para vectorizar con bag of words, utilizaremos sklearn.

En sklearn, este elemento se llama CountVectorizer.

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html


Sigue la lógica de fit/transform.

Podemos ver algunos parámetros importantes como por ejemplo:

- ngram_range
- lowercase
- stop_words
- strip_accents


Antes de aplicar count vectorizer sobre nuestro df, vamos a aplicarle nuestra función "clean_text".

EJERCICIO: Aplicar clean_text a todo nuestro dataframe.

Este proceso puede tomar más de media hs para el dataset que tenemos, por lo tanto, nos quedaremos únicamente con las primeras 5mil filas para poder ejecutar el código en clases. Luego ustedes pueden probarlo con el dataset completo.

In [33]:
# COMENTAR ESTA CELDA SI QUIEREN TRABAJAR CON EL DATASET COMPLETO (les puede tomar 30 min el preprocesamiento)
pos_samples = df[df.TARGET=='POS'].head(2500)
neg_samples = df[df.TARGET=='NEG'].head(2500)

df = pd.concat([pos_samples, neg_samples])

In [34]:
df.shape

(5000, 2)

In [35]:
%%time
df["REVIEW"] = df["REVIEW"].apply(clean_text)

CPU times: user 4min 41s, sys: 446 ms, total: 4min 41s
Wall time: 5min 18s


Ahora, debemos hacer train_test_split. Utilizar como random_state 0 y test_size de 0.2

In [36]:
#COMPLETAR
from sklearn.model_selection import train_test_split
X=df["REVIEW"]
y=df["TARGET"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Ahora si, importemos count vectorizer y lo apliquemos en nuestro texto.

Recuerden: fit solo sobre train.

In [37]:
from sklearn.feature_extraction.text import CountVectorizer

In [38]:
#COMPLETAR
cv = CountVectorizer()
X_train = cv.fit_transform(X_train)
X_test = cv.transform(X_test)

In [39]:
X_train

<4000x29548 sparse matrix of type '<class 'numpy.int64'>'
	with 345402 stored elements in Compressed Sparse Row format>

In [40]:
X_test

<1000x29548 sparse matrix of type '<class 'numpy.int64'>'
	with 86146 stored elements in Compressed Sparse Row format>

Ahora, además podemos ver las "features" con la siguiente función de nuestro count vectorizer (en la siguiente celda, cambien el nombre del vectorizer que ustedes hayan utilizado).

In [41]:
cv.get_feature_names_out()

array(['00', '000', '000s', ..., 'étc', 'ísnt', 'über'], dtype=object)

Vamos a encontrar muchisimas palabras que no tienen sentido y no aportan nada a nuestro modelo, o caracteres como "__________________________________________________________________".
Todo esto podríamos tenerlo en cuenta para la etapa de preprocesamiento.

Con el X_train y X_test que generamos con nuestro countVectorizer ya podríamos entrenar un modelo.

Ahora, aplicaremos TF IDF


### TFIDF

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

Generar X_train, X_test, y_train e y_test de nuevo, ya que las modificamos anteriormente con el count vectorizer:

In [42]:
#COMPLETAR
from sklearn.model_selection import train_test_split
X=df["REVIEW"]
y=df["TARGET"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Ahora, importar tfidf vectorizer y aplicarlo sobre X_train y X_test

In [43]:
#COMPLETAR
from sklearn.feature_extraction.text import TfidfVectorizer
tdif = TfidfVectorizer()
X_train = tdif.fit_transform(X_train)
X_test = tdif.transform(X_test)

In [44]:
X_train

<4000x29548 sparse matrix of type '<class 'numpy.float64'>'
	with 345402 stored elements in Compressed Sparse Row format>

In [45]:
X_test

<1000x29548 sparse matrix of type '<class 'numpy.float64'>'
	with 86146 stored elements in Compressed Sparse Row format>

Entrenar un SVC con los datos ya vectorizados.

Utilizaremos:
- random_state=0
- C=0.5

In [46]:
#COMPLETAR
from sklearn.svm import SVC
svc = SVC(random_state=0, C=0.5)
svc.fit(X_train, y_train)

SVC(C=0.5, random_state=0)

In [47]:
y_train

413     NEG
775     NEG
775     POS
217     POS
1245    POS
       ... 
2431    NEG
764     NEG
1653    POS
107     NEG
232     NEG
Name: TARGET, Length: 4000, dtype: object

Ahora, como siempre hicimos, podemos medir métricas. Por ejemplo, imprimir el classification report.

In [49]:
#COMPLETAR
from sklearn.metrics import classification_report
print(classification_report(y_train, svc.predict(X_train)))
print(classification_report(y_test, svc.predict(X_test)))

              precision    recall  f1-score   support

         NEG       0.99      0.96      0.97      1981
         POS       0.96      0.99      0.97      2019

    accuracy                           0.97      4000
   macro avg       0.97      0.97      0.97      4000
weighted avg       0.97      0.97      0.97      4000

              precision    recall  f1-score   support

         NEG       0.90      0.74      0.81       519
         POS       0.76      0.91      0.83       481

    accuracy                           0.82      1000
   macro avg       0.83      0.83      0.82      1000
weighted avg       0.84      0.82      0.82      1000

